In [3]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import networkx as nx
from collections import defaultdict
import math
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()


df=pd.read_excel("mmdCaseFile.xlsx")

x_col=df.columns[2:-1]
y=df['MaternalMortalityDeath_721']
x=df[x_col]

x=sm.add_constant(x)
model=sm.OLS(y,x).fit()
target=10

coef=model.params.values[1:]
alpha = [0.5] * 4
alpha = np.array(alpha)
x.drop('const',axis=1,inplace=True)
x- alpha*(y.mean()-target)/coef
x_new =(x-alpha*(y.mean()-target)/coef)
x_new=sm.add_constant(x_new)

def clip(col):
    
  return np.clip(col,0,100)

x_new=x_new.apply(clip)
y_new=x_new.dot(model.params.values)

y_new=np.clip(y_new,0,700)
x_new.drop('const',axis=1,inplace=True)

x_new['Maternal Deaths']=y_new

Loading BokehJS ...

In [4]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

def init_graph_attr(G, AdjFile, values):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    # Handle NaN values by replacing them with 0
    values = values.fillna(0)
    
    # Use the provided 'values' directly
    capability_vector = list(zip(*[values]))
    
    node_attri_dict = dict(zip(node_list[:30], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1
def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1

def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

def get_node_stability(G,dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [5]:
dim = 1
G = nx.Graph()
adjacency_file = "KAG 2016-17/Karnataka_District_Adjacency_File.xlsx"

def calculate_and_map_stability(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()
    scaler = MinMaxScaler()
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))

    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")
    init_graph_attr(G, adjacency_file, column_to_pass)

    stability_dict = get_node_stability(G, dim)
    stability_column = df["District"].map(stability_dict)

    return stability_column

In [6]:
# x_new

In [7]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.models import HoverTool

def update_x_new(x_new, alphas, target):
    x_new_copy = x_new.copy()
    columns_to_update = x_new_copy.columns.difference(['Maternal Deaths', 'New Stability'])

    if 'const' in columns_to_update:
        columns_to_update = columns_to_update.drop('const')
    
    alphas_array = np.array(alphas)  # Convert alphas to a numpy array
    x_new_copy[columns_to_update] -= alphas_array * (x_new['Maternal Deaths'].mean() - target) / coef
    x_new_copy[columns_to_update] = x_new_copy[columns_to_update].apply(clip)
    
    y_new = x_new_copy.dot(model.params.values)
    y_new = np.clip(y_new, 0, 700)
    x_new_copy['Maternal Deaths'] = y_new
    x_new["New Stability"] = np.nan

    values = x_new["Maternal Deaths"].values
    result_column = calculate_and_map_stability(G, df, adjacency_file, values, dim)
    x_new["New Stability"] = result_column
    x_new["New Stability"] = pd.to_numeric(x_new["New Stability"])


alpha_values_default = [0.5] * 4
target_value_default = 15
update_x_new(x_new, alpha_values_default, target_value_default)


In [13]:
x_new

,NFHS_%BirthsDeliveredInAHealthFacility,NFHS_%BirthsDeliveredByCaesareanSection,Identified_Leprosy_Patients_Female_700,Death_TB_Patients_Female_712,Maternal Deaths,New Stability
0,100.000000,41.72172,40.344258,59.935629,17.843723,0.353207
1,100.000000,44.72172,0.000000,0.000000,3.139506,0.801263
2,100.000000,41.52172,0.000000,0.000000,4.272046,0.843947
3,100.000000,42.32172,0.000000,0.000000,3.988911,0.828716
4,100.000000,41.92172,0.000000,0.000000,4.130478,0.887349
5,100.000000,46.02172,0.000000,0.000000,2.679412,0.732058
6,100.000000,39.32172,0.000000,0.000000,5.050667,0.981607
7,100.000000,45.52172,0.000000,0.000000,2.856371,0.956815
8,100.000000,47.62172,0.000000,36.935629,8.097790,0.771589
9,100.000000,54.52172,0.000000,0.000000,0.000000,0.840707


In [8]:
# def create_bokeh_plot(x_new, alpha_values, target_value):
#     # Calculate initial stability
#     initial_stability = calculate_and_map_stability(G, df, adjacency_file, x_new["Maternal Deaths"].values, dim)

#     # Create a ColumnDataSource
#     source = ColumnDataSource(data=dict(District=df["District"], Initial_Stability=initial_stability, New_Stability=x_new["New Stability"]))

#     # Create a Bokeh plot
#     plot = figure(x_range=df["District"], height=350, title="Stability Comparison", toolbar_location=None, tools="")
#     plot.vbar(x='District', top='Initial_Stability', width=0.9, legend_label='Initial Stability', line_color='white', source=source, color='orange')
#     plot.vbar(x='District', top='New_Stability', width=0.9, legend_label='New Stability', line_color='white', source=source, color='blue')

#     # Add HoverTool for tooltips
#     hover = HoverTool()
#     hover.tooltips = [("District", "@District"), ("Initial Stability", "@Initial_Stability"), ("New Stability", "@New_Stability")]
#     plot.add_tools(hover)

#     # Set up sliders for alpha values and target value
#     alpha_sliders = [Slider(title=f'Alpha {i}', value=alpha, start=0, end=1, step=0.01) for i, alpha in enumerate(alpha_values)]
#     target_slider = Slider(title='Target Value', value=target_value, start=0, end=100, step=1)

#     # Define callback function for slider changes
#     def update_data(attrname, old, new):
#         # Update alpha values and target value
#         alphas = [slider.value for slider in alpha_sliders]
#         target = target_slider.value

#         # Update x_new_all_comb with new alpha values and target value
#         update_x_new(x_new, alphas, target)

#         # Calculate and update New Stability
#         new_stability = calculate_and_map_stability(G, df, adjacency_file, x_new["Maternal Deaths"].values, dim)
#         source.data = dict(District=df["District"], Initial_Stability=initial_stability, New_Stability=new_stability)

#     # Attach callback function to slider changes
#     for slider in alpha_sliders + [target_slider]:
#         slider.on_change('value', update_data)

#     # Layout setup
#     layout = column(plot, *alpha_sliders, target_slider)
#     curdoc().add_root(layout)
#     return layout

In [9]:
# # ...

# def create_bokeh_plot(x_new, alpha_values, target_value):
#     # Calculate initial stability
#     initial_stability = calculate_and_map_stability(G, df, adjacency_file, x_new["Maternal Deaths"].values, dim)

#     # Create a ColumnDataSource
#     source = ColumnDataSource(data=dict(District=df["District"], Initial_Stability=initial_stability, New_Stability=x_new["New Stability"]))

#     # Create a Bokeh plot
#     plot = figure(x_range=df["District"], height=350, title="Stability Comparison", toolbar_location=None, tools="")
#     plot.vbar(x='District', top='Initial_Stability', width=0.9, legend_label='Initial Stability', line_color='white', source=source, color='orange')
#     plot.vbar(x='District', top='New_Stability', width=0.9, legend_label='New Stability', line_color='white', source=source, color='blue')

#     # Add HoverTool for tooltips
#     hover = HoverTool()
#     hover.tooltips = [("District", "@District"), ("Initial Stability", "@Initial_Stability"), ("New Stability", "@New_Stability")]
#     plot.add_tools(hover)

#     # Set up sliders for alpha values and target value
#     alpha_sliders = [Slider(title=f'Alpha {i}', value=alpha, start=0, end=1, step=0.01) for i, alpha in enumerate(alpha_values)]
#     target_slider = Slider(title='Target Value', value=target_value, start=0, end=100, step=1)

#     # Define callback function for slider changes
#     def update_data(attrname, old, new):
#         # Update alpha values and target value
#         alphas = [slider.value for slider in alpha_sliders]
#         target = target_slider.value

#         # Update x_new_all_comb with new alpha values and target value
#         update_x_new(x_new, alphas, target)

#         # Calculate and update New Stability
#         new_stability = calculate_and_map_stability(G, df, adjacency_file, x_new["Maternal Deaths"].values, dim)
#         source.data = dict(District=x_new["District"], Initial_Stability=initial_stability, New_Stability=new_stability)

#     # Attach callback function to slider changes
#     for slider in alpha_sliders + [target_slider]:
#         slider.on_change('value', update_data)

#     # Layout setup
#     layout = column(plot, *alpha_sliders, target_slider)
#     curdoc().add_root(layout)
#     show(layout)

# create_bokeh_plot(x_new, alpha_values_default, target_value_default)


In [10]:
# create_bokeh_plot(x_new, alpha_values_default, target_value_default)
# 

In [11]:
# import pandas as pd
# import statsmodels.api as sm
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import networkx as nx
# from collections import defaultdict
# import math
# from sklearn.preprocessing import MinMaxScaler
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models import HoverTool, ColumnDataSource
# from bokeh.palettes import Category20
# from bokeh.server.server import Server
# from tornado.ioloop import IOLoop

# df=pd.read_excel("mmdCaseFile.xlsx")

# x_col=df.columns[2:-1]
# y=df['MaternalMortalityDeath_721']
# x=df[x_col]

# x=sm.add_constant(x)
# model=sm.OLS(y,x).fit()
# target=10

# coef=model.params.values[1:]
# alpha = [0.5] * 4
# alpha = np.array(alpha)
# x.drop('const',axis=1,inplace=True)
# x- alpha*(y.mean()-target)/coef
# x_new =(x-alpha*(y.mean()-target)/coef)
# x_new=sm.add_constant(x_new)

# def clip(col):
    
#   return np.clip(col,0,100)

# x_new=x_new.apply(clip)
# y_new=x_new.dot(model.params.values)

# y_new=np.clip(y_new,0,700)
# x_new.drop('const',axis=1,inplace=True)

# x_new['Maternal Deaths']=y_new

# def init_graph(G,node_adj_frame):
#     G.add_nodes_from([i for i in range(len(node_adj_frame))])
#     labels = {}
#     labels = node_adj_frame.columns
#     for i in range(len(node_adj_frame)):
#         snode = node_adj_frame[labels[0]][i]-1
#         temp = node_adj_frame[labels[2]][i]
#         if ',' in str(temp):
#             sedge_arr = temp.split(',')
#             for j in range(0, len(sedge_arr)):
#                 k = int(sedge_arr[j])
#                 G.add_edge(snode, k-1)
#         elif np.isnan(temp):
#             print("ERROR: Not found in the adjacency excel sheet")
#         else:
#             G.add_edge(snode, int(temp)-1)
#     return

# def init_graph_attr(G, AdjFile, values):
#     node_adj_frame = pd.read_excel(AdjFile)
#     node_list = node_adj_frame["District_Name"].tolist()
#     node_list.insert(80, "")
#     nodeAttr = {}
#     init_graph(G, node_adj_frame)
    
#     # Handle NaN values by replacing them with 0
#     values = values.fillna(0)
    
#     # Use the provided 'values' directly
#     capability_vector = list(zip(*[values]))
    
#     node_attri_dict = dict(zip(node_list[:30], capability_vector))
#     node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

#     for i in range(len(node_adj_frame)):
#         temp = {}
#         temp["capabilityvector"] = node_attri_dict[node_list[i]]
#         temp["nodeStress"] = 0
#         temp["name"] = node_list[i]
#         nodeAttr[i] = temp

#     nx.set_node_attributes(G, nodeAttr)

# def addList(l1,l2):
#     for i in range(len(l1)):
#         l1[i] = l1[i] + l2[i]
#     return l1
# def divList(l1,k):
#     for i in range(len(l1)):
#         l1[i] = l1[i]/k
#     return l1

# def l2_normalization(l1,l2):
#     k = 0
#     for i in range(len(l1)):
#         k+= (l1[i] - l2[i])**2
#     return math.sqrt(k)

# def get_node_stability(G,dim):
#     taluka_stress_dict = {}
#     for n in G.nodes():
#         centroid = [0]*dim
#         neighList = list(G.neighbors(n))
#         for nei in neighList:
#             try:
#                 centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
#             except(KeyError):
#                 pass
#         try:
#             G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
#         except(KeyError):
#             pass
#         try:
#             taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
#         except(KeyError):
#             pass
#     return taluka_stress_dict

# dim = 1
# G = nx.Graph()
# adjacency_file = "KAG 2016-17/Karnataka_District_Adjacency_File.xlsx"

# def calculate_and_map_stability(G, df, adjacency_file, column_values, dim):
#     G = nx.Graph()
#     scaler = MinMaxScaler()
#     normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))

#     column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")
#     init_graph_attr(G, adjacency_file, column_to_pass)

#     stability_dict = get_node_stability(G, dim)
#     stability_column = df["District"].map(stability_dict)

#     return stability_column

# from bokeh.layouts import column
# from bokeh.models import ColumnDataSource, Slider
# from bokeh.io import curdoc
# from bokeh.plotting import figure
# from bokeh.models import HoverTool

# def update_x_new(x_new, alphas, target):
#     x_new_copy = x_new.copy()
#     print(x_new_copy.columns)
#     columns_to_update = x_new_copy.columns.difference(['Maternal Deaths', 'New Stability'])
#     print(x_new_copy.columns)
#     if 'const' in columns_to_update:
#         columns_to_update = columns_to_update.drop('const')
    
#     alphas_array = np.array(alphas)  # Convert alphas to a numpy array
#     x_new_copy[columns_to_update] -= alphas_array * (x_new['Maternal Deaths'].mean() - target) / coef
#     x_new_copy[columns_to_update] = x_new_copy[columns_to_update].apply(clip)
    
#     y_new = x_new_copy.dot(model.params.values)
#     y_new = np.clip(y_new, 0, 700)
#     x_new_copy['Maternal Deaths'] = y_new
#     x_new["New Stability"] = np.nan

#     values = x_new["Maternal Deaths"].values
#     result_column = calculate_and_map_stability(G, df, adjacency_file, values, dim)
#     x_new["New Stability"] = result_column
#     x_new["New Stability"] = pd.to_numeric(x_new["New Stability"])


# alpha_values_default = [0.5] * 4
# target_value_default = 10
# update_x_new(x_new, alpha_values_default, target_value_default)

# # Function to create the Bokeh plot
# def create_bokeh_plot():
#     # Calculate initial stability
#     dim  = 1
#     initial_stability = calculate_and_map_stability(G,df, adjacency_file, x_new["Maternal Deaths"].values, dim)

#     # Create a ColumnDataSource
#     source = ColumnDataSource(data=dict(District=df["District"], Initial_Stability=initial_stability, New_Stability=x_new["New Stability"]))

#     # Create a Bokeh plot
#     plot = figure(x_range=df["District"], height=350, title="Stability Comparison", toolbar_location=None, tools="")
#     plot.vbar(x='District', top='Initial_Stability', width=0.9, legend_label='Initial Stability', line_color='white', source=source, color='orange')
#     plot.vbar(x='District', top='New_Stability', width=0.9, legend_label='New Stability', line_color='white', source=source, color='blue')

#     # Add HoverTool for tooltips
#     hover = HoverTool()
#     hover.tooltips = [("District", "@District"), ("Initial Stability", "@Initial_Stability"), ("New Stability", "@New_Stability")]
#     plot.add_tools(hover)

#     # Set up sliders for alpha values and target value
#     alpha_sliders = [Slider(title=f'Alpha {i}', value=alpha, start=0, end=1, step=0.01) for i, alpha in enumerate(alpha_values_default)]
#     target_slider = Slider(title='Target Value', value=target_value_default, start=0, end=100, step=1)

#     # Define callback function for slider changes
#     def update_data(attrname, old, new):
#         # Update alpha values and target value
#         alphas = [slider.value for slider in alpha_sliders]
#         target = target_slider.value

#         # Update x_new with new alpha values and target value
#         update_x_new(x_new, alphas, target)

#         # Calculate and update New Stability
#         new_stability = calculate_and_map_stability(G,df, adjacency_file, x_new["Maternal Deaths"].values, dim)
#         source.data = dict(District=df["District"], Initial_Stability=initial_stability, New_Stability=new_stability)

#     # Attach callback function to slider changes
#     for slider in alpha_sliders + [target_slider]:
#         slider.on_change('value', update_data)

#     # Layout setup
#     layout = column(plot, *alpha_sliders, target_slider)
#     show(layout)

In [12]:
# curdoc().add_root(create_bokeh_plot())
# 